In [6]:
pip install python-docx 

  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184508 sha256=e62172a5e95edd4c64f49d2d3fecefa842c83af5f31d191ea7c018b08f267411
  Stored in directory: c:\users\numam\appdata\local\pip\cache\wheels\83\8b\7c\09ae60c42c7ba4ed2dddaf2b8b9186cb105255856d6ed3dba5
Successfully built python-docx
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from googletrans import Translator
import language_tool_python
import PyPDF2
import docx
import regex as re
import string
import PyPDF2
import spacy
import torch
from string import punctuation
from numba import jit, cuda
from deepmultilingualpunctuation import PunctuationModel
from newspaper import Article
from spacy.lang.en.stop_words import STOP_WORDS
from transformers import (
    PegasusForConditionalGeneration, PegasusTokenizer, pipeline)
from heapq import nlargest
#from GPUtil import showUtilization as gpu_usage
from numba import cuda


# ----------------------------------------------------------------
# Translators
# ----------------------------------------------------------------


def translate(summaryText, lang_code):
    translator = Translator()
    Transtxt = translator.translate(summaryText, src='en', dest=lang_code)
    return Transtxt


# ----------------------------------------------------------------
#  Convert
# ----------------------------------------------------------------
def filePDF_extract(filePath):

    pdffileobj = open(filePath, 'rb')
    pdfreader = PyPDF2.PdfFileReader(pdffileobj)
    x = pdfreader.numPages
    content = ""
    RetText = ""
    for i in range(x):
        pageobj = pdfreader.getPage(i)
        content = pageobj.extractText()
        RetText = RetText + content
    return RetText


def fileDOCX_extract(filePath):
    doc = docx.Document(filePath)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)


def fileTXT_extract(filepath):
    TextFile = open(filepath, "r")
    textRead = TextFile.read()
    TextFile.close()
    return textRead


def URL_extract(url,url1):

    link_paper = (url)
    article = Article(link_paper)
    article.download()
    article.parse()
    input_text = article.text
    input_text = ' '.join(input_text.split())
    link_paper1 = (url1)
    article1 = Article(link_paper1)
    article1.download()
    article1.parse()
    input_text1 = article1.text
    input_text1 = ' '.join(input_text1.split())
    #total_text=input_text + input_text1
    #print(total_text)
    #return total_text

    clean_text = str(input_text.lower())
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(clean_text)
    per = 1.0
    tokens = [token.text for token in doc]

    word_frequencies = {}

    for word in doc:

        if word.text.lower() not in list(STOP_WORDS):

            if word.text.lower() not in punctuation:

                if word.text not in word_frequencies.keys():

                    word_frequencies[word.text] = 1

                else:

                    word_frequencies[word.text] += 1

    max_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}

    for sent in sentence_tokens:

        for word in sent:

            if word.text.lower() in word_frequencies.keys():

                if sent not in sentence_scores.keys():

                    sentence_scores[sent] = word_frequencies[word.text.lower()]

                else:

                    sentence_scores[sent] += word_frequencies[word.text.lower()]
    print(sentence_scores)
    select_length = int(len(sentence_tokens) * per)
    print(select_length)
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    # print(summary)
    final_summary = [word.text for word in summary]
    # print(final_summary)
    spacy_summary = ' '.join(final_summary)

    clean_text = str(input_text1.lower())
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(clean_text)
    per = 1.0
    tokens = [token.text for token in doc]

    word_frequencies = {}

    for word in doc:

        if word.text.lower() not in list(STOP_WORDS):

            if word.text.lower() not in punctuation:

                if word.text not in word_frequencies.keys():

                    word_frequencies[word.text] = 1

                else:

                    word_frequencies[word.text] += 1

    max_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}

    for sent in sentence_tokens:

        for word in sent:

            if word.text.lower() in word_frequencies.keys():

                if sent not in sentence_scores.keys():

                    sentence_scores[sent] = word_frequencies[word.text.lower()]

                else:

                    sentence_scores[sent] += word_frequencies[word.text.lower()]
    print(sentence_scores)
    select_length = int(len(sentence_tokens) * per)
    print(select_length)
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    # print(summary)
    final_summary1= [word.text for word in summary]
    # print(final_summary)
    spacy_summary = ' '.join(final_summary)

    summ=str(final_summary+final_summary1)
    print("hello")
    print(summ)
    return summ



def setGrammer(inputText):
    model = PunctuationModel()

    inputText = inputText.replace("<n>", "")

    inputText = model.restore_punctuation(inputText)

    my_tool = language_tool_python.LanguageTool('en-US')

    inputText = my_tool.correct(inputText)

    return inputText


def Clean_Text(textforCleanup):
    text = re.sub("https?:\/\/.*[\r\n]*", "", textforCleanup)
    text = re.sub("#", "", text)
    punct = set(string.punctuation)
    text = "".join([ch for ch in text if ch not in punct])
    text = text.encode(encoding="ascii", errors="ignore")
    text = text.decode()
    clean_text = " ".join([word for word in text.split()])
    clean_text = str(clean_text.lower())
    return clean_text


# ----------------------------------------------------------------
# BASIC ALGO
# ----------------------------------------------------------------


def PegasusModel(spacy_summary):
    #torch.cuda.empty_cache()
    #torch.cuda.memory_summary(device=None, abbreviated=False)
    #freeMemory()
    model_name = "google/pegasus-cnn_dailymail"

    # Load pretrained tokenizer
    # device = "cuda" if torch.cuda.is_available() else "cpu"
    device = "cpu"

    pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)

    pegasus_model = PegasusForConditionalGeneration.from_pretrained(
        model_name).to(device)

    # Create tokens

    tokens = pegasus_tokenizer(
        spacy_summary, truncation=True, padding="max_length", return_tensors="pt").to(device)

    encoded_summary = pegasus_model.generate(**tokens)

    # Decode summarized text

    decoded_summary = pegasus_tokenizer.decode(

        encoded_summary[0],

        skip_special_tokens=True

    )

    # print(decoded_summary)

    summarizer = pipeline(

        "summarization",

        model=model_name,

        tokenizer=pegasus_tokenizer,

        framework="pt", truncation=True

    )

    summary = summarizer(spacy_summary, min_length=30, max_length=150)

    summary = summary[0]["summary_text"]

    summary = summary.replace("<n>", "")

    summary = summary.replace("  ", "")


    return summary


def Spacy(input_text):
    clean_text = str(input_text.lower())
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(clean_text)
    per = 1.0
    tokens = [token.text for token in doc]

    word_frequencies = {}

    for word in doc:

        if word.text.lower() not in list(STOP_WORDS):

            if word.text.lower() not in punctuation:

                if word.text not in word_frequencies.keys():

                    word_frequencies[word.text] = 1

                else:

                    word_frequencies[word.text] += 1

    max_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / max_frequency

    sentence_tokens = [sent for sent in doc.sents]
    sentence_scores = {}

    for sent in sentence_tokens:

        for word in sent:

            if word.text.lower() in word_frequencies.keys():

                if sent not in sentence_scores.keys():

                    sentence_scores[sent] = word_frequencies[word.text.lower()]

                else:

                    sentence_scores[sent] += word_frequencies[word.text.lower()]
    print(sentence_scores)
    select_length = int(len(sentence_tokens) * per)
    print(select_length)
    summary = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    # print(summary)
    final_summary = [word.text for word in summary]
    #print(final_summary)
    spacy_summary = ' '.join(final_summary)
    print(spacy_summary)

    return spacy_summary


# def freeMemory():
#
#     print("Initial GPU Usage")
#    # gpu_usage()
#
#     torch.cuda.empty_cache()
#
#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)
#
#     print("GPU Usage after emptying the cache")
#     #gpu_usage()


print(PegasusModel(Clean_Text(URL_extract("https://www.thehindu.com/news/cities/Mangalore/ngt-directs-kspcb-to-prevent-river-water-pollution-around-mangaluru-sets-timeline/article66173090.ece","https://economictimes.indiatimes.com/news/india/delhi-air-pollution-aqi-back-in-very-poor-category-construction-and-demolition-work-still-banned/videoshow/95653899.cms"))))

{november 23, 2022 01:51 pm | updated 01:57 pm ist - mangaluru principal bench of the national green tribunal (ngt) in new delhi has directed karnataka state pollution control board (kspcb) and the state government to frame an action plan for remedial action to prevent environment pollution through sewage and solid waste entering the natural water system in and around mangaluru within a month.: 16.66666666666666, the order was passed on monday, november 21 thereafter, the joint committee, constituted earlier on the direction of the tribunal on april 29, 2022, execute the action plan within a month.: 6.833333333333333, the status of compliance as on january 31, 2023, should be reported to the tribunal latest by february 15, according to the tribunal’s bench comprising chairman adarsh kumar goel, judicial member arun kumar tyagi and expert members a. senthil vel and afroz ahmad.: 7.333333333333337, hearing original application no. 307/2022 taken up suo motu following a report published b

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]